In [26]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
import time, datetime
from utils import *
from tqdm import tqdm

modified 0330

# 1. statistics overview

In [11]:
bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment12/2012-2013-data-with-predictions-4-final.csv'
log = pd.read_csv(bath_path, encoding = "utf-8",low_memory=False)

print('The columns of log are ', log.columns)
print('The length of log is {}'.format(len(log)))

print('###########################################')

print(count_unique(log, log.columns))

The columns of log are  Index(['problem_log_id', 'skill', 'problem_id', 'user_id', 'assignment_id',
       'assistment_id', 'start_time', 'end_time', 'problem_type', 'original',
       'correct', 'bottom_hint', 'hint_count', 'actions', 'attempt_count',
       'ms_first_response', 'tutor_mode', 'sequence_id', 'student_class_id',
       'position', 'type', 'base_sequence_id', 'skill_id', 'teacher_id',
       'school_id', 'overlap_time', 'template_id', 'answer_id', 'answer_text',
       'first_action', 'problemlogid', 'Average_confidence(FRUSTRATED)',
       'Average_confidence(CONFUSED)', 'Average_confidence(CONCENTRATING)',
       'Average_confidence(BORED)'],
      dtype='object')
The length of log is 6123270
###########################################
number of unique value in problem_log_id: 6123270
number of unique value in skill: 199
number of unique value in problem_id: 179999
number of unique value in user_id: 46674
number of unique value in assignment_id: 189760
number of unique

In [13]:
for col in log.columns:
    print(col, log[col][0:5])

problem_log_id 0    137792159
1    138083797
2    142332619
3    145939397
4    137111284
Name: problem_log_id, dtype: int64
skill 0                                     NaN
1                                Rounding
2    Multiplication and Division Integers
3                              Proportion
4                                     NaN
Name: skill, dtype: object
problem_id 0    557460
1    365981
2    426415
3     86686
4    399669
Name: problem_id, dtype: int64
user_id 0    61394
1    61394
2    61394
3    61394
4    76592
Name: user_id, dtype: int64
assignment_id 0    565736
1    573819
2    734130
3    821352
4    557216
Name: assignment_id, dtype: int64
assistment_id 0    341511
1    204043
2    247525
3     48081
4    227869
Name: assistment_id, dtype: int64
start_time 0    2012-09-28 15:11:27
1    2012-10-09 11:01:52
2    2013-03-07 10:53:20
3    2013-08-20 19:54:56
4    2012-09-10 17:20:10
Name: start_time, dtype: object
end_time 0    2012-09-28 15:11:36.856
1    2012-10-09 1

In [14]:
interested_col = [
    'user_id', 'problem_id', 'skill_id', 'start_time', 'end_time', 'correct', 
    'skill', 'problem_type', 'answer_text', 
]

In [15]:
log.head()

,problem_log_id,skill,problem_id,user_id,assignment_id,assistment_id,start_time,end_time,problem_type,original,...,overlap_time,template_id,answer_id,answer_text,first_action,problemlogid,Average_confidence(FRUSTRATED),Average_confidence(CONFUSED),Average_confidence(CONCENTRATING),Average_confidence(BORED)
0,137792159,NaN,557460,61394,565736,341511,2012-09-28 15:11:27,2012-09-28 15:11:36.856,choose_1,1,...,9852,341511,NaN,she,0,137792159,0.361323,0.0,0.336529,0.000000
1,138083797,Rounding,365981,61394,573819,204043,2012-10-09 11:01:52,2012-10-09 11:02:13.182,algebra,1,...,21175,204043,NaN,74.29,0,138083797,0.361323,0.0,0.766925,0.000000
2,142332619,Multiplication and Division Integers,426415,61394,734130,247525,2013-03-07 10:53:20,2013-03-07 10:53:28.661,algebra,1,...,8645,247525,NaN,00,0,142332619,0.361323,0.0,0.766925,0.442968
3,145939397,Proportion,86686,61394,821352,48081,2013-08-20 19:54:56,2013-08-20 19:55:21.753,algebra,1,...,25728,46362,NaN,3.8,0,145939397,0.775000,0.0,0.766925,0.912281
4,137111284,NaN,399669,76592,557216,227869,2012-09-10 17:20:10,2012-09-10 17:24:56.579,choose_1,1,...,286578,227869,NaN,C (wr - 1)(wr + 1),0,137111284,0.361323,0.0,0.766925,0.000000


In [16]:
check_nan(log)

number of NaN in column problem_log_id: 0
number of NaN in column skill: 3493190
number of NaN in column problem_id: 0
number of NaN in column user_id: 0
number of NaN in column assignment_id: 0
number of NaN in column assistment_id: 0
number of NaN in column start_time: 0
number of NaN in column end_time: 0
number of NaN in column problem_type: 0
number of NaN in column original: 0
number of NaN in column correct: 0
number of NaN in column bottom_hint: 60348
number of NaN in column hint_count: 0
number of NaN in column actions: 0
number of NaN in column attempt_count: 0
number of NaN in column ms_first_response: 0
number of NaN in column tutor_mode: 0
number of NaN in column sequence_id: 0
number of NaN in column student_class_id: 0
number of NaN in column position: 0
number of NaN in column type: 0
number of NaN in column base_sequence_id: 0
number of NaN in column skill_id: 3411457
number of NaN in column teacher_id: 0
number of NaN in column school_id: 157
number of NaN in column o

In [5]:
count_unique(log, ['user_id', 'skill_id', 'problem_id'])
print(len(log))

number of NaN in column problem_log_id: 0
number of NaN in column skill: 3493190
number of NaN in column problem_id: 0
number of NaN in column user_id: 0
number of NaN in column assignment_id: 0
number of NaN in column assistment_id: 0
number of NaN in column start_time: 0
number of NaN in column end_time: 0
number of NaN in column problem_type: 0
number of NaN in column original: 0
number of NaN in column correct: 0
number of NaN in column bottom_hint: 60348
number of NaN in column hint_count: 0
number of NaN in column actions: 0
number of NaN in column attempt_count: 0
number of NaN in column ms_first_response: 0
number of NaN in column tutor_mode: 0
number of NaN in column sequence_id: 0
number of NaN in column student_class_id: 0
number of NaN in column position: 0
number of NaN in column type: 0
number of NaN in column base_sequence_id: 0
number of NaN in column skill_id: 3411457
number of NaN in column teacher_id: 0
number of NaN in column school_id: 157
number of NaN in column o

In [26]:
log['correct'].mean()

0.6768206326031685

# 2. remove learner logs with threshold

In [6]:
remove_thres = [50, 100, 150, 200]

for thres in remove_thres:
    print('remove threshold is {}'.format(thres))
    log = log[~log.skill_id.isna()]
    log = remove_log(log, 'user_id', num=thres)
    count_unique(log, ['user_id', 'skill_id', 'problem_id'])
    print(len(log))

remove threshold is 50
number of unique value in user_id: 12443
number of unique value in skill_id: 263
number of unique value in problem_id: 51866
2431788
remove threshold is 100
number of unique value in user_id: 7543
number of unique value in skill_id: 260
number of unique value in problem_id: 50791
2081199
remove threshold is 150
number of unique value in user_id: 5076
number of unique value in skill_id: 255
number of unique value in problem_id: 49209
1779709
remove threshold is 200
number of unique value in user_id: 3696
number of unique value in skill_id: 251
number of unique value in problem_id: 47426
1542919


In [21]:
# should reload log here
remove_thres = [50, 100, 150, 200]
for thres in remove_thres:
    print(f'Remove threshold is {thres}')

    # Remove rows with NaN skill_id and users who appear less than thres times
    log = log[~log.skill_id.isna()]
    log = remove_log(log, 'user_id', num=thres)

    # Create new DataFrame with desired columns
    df = log[['user_id', 'problem_id', 'skill_id', 'correct', 'skill', 'problem_type', 'answer_text']].copy()
    df.columns = ['user_id', 'problem_id', 'skill_id', 'correct', 'skill_text', 'problem_text', 'answer_text']

    # Convert timestamp columns to a single timestamp value
    time_format = "%Y-%m-%d %H:%M:%S"
    start_times = pd.to_datetime(log.start_time, format=time_format).apply(lambda x: x.timestamp())
    end_times = pd.to_datetime(log.end_time, format=time_format).apply(lambda x: x.timestamp())
    df['timestamp'] = (start_times + end_times) / 2
    df['dwell_time'] = end_times - start_times

    # Re-index columns
    df = df.astype({'timestamp': np.float64, 'dwell_time': np.float64, 'correct': np.float64})
    df = df.astype({'problem_id': np.int64, 'user_id': np.int64, 'skill_id': np.int64})
    df['skill_id'] = pd.Categorical(df['skill_id'], categories=df['skill_id'].unique()).codes
    df['problem_id'] = pd.Categorical(df['problem_id'], categories=df['problem_id'].unique()).codes
    df['user_id'] = pd.Categorical(df['user_id'], categories=df['user_id'].unique()).codes

    # Save cleaned DataFrame to a CSV file
    df.to_csv(f'/mnt/qb/work/mlcolab/hzhou52/kt/assistment12/interactions_{thres}.csv', sep='\t', index=False)

    print(f'Remove threshold {thres} is done!')
    print(count_unique(df, df.columns.tolist()))
    print(f'Length of DataFrame: {len(df)}')

Remove threshold is 50
Remove threshold 50 is done!
number of unique value in user_id: 12443
number of unique value in problem_id: 51866
number of unique value in skill_id: 263
number of unique value in correct: 8
number of unique value in skill_text: 198
number of unique value in problem_text: 6
number of unique value in answer_text: 104219
number of unique value in timestamp: 2431626
number of unique value in dwell_time: 376678
None
Length of DataFrame: 2431788
Remove threshold is 100
Remove threshold 100 is done!
number of unique value in user_id: 7543
number of unique value in problem_id: 50791
number of unique value in skill_id: 260
number of unique value in correct: 6
number of unique value in skill_text: 198
number of unique value in problem_text: 6
number of unique value in answer_text: 94560
number of unique value in timestamp: 2081079
number of unique value in dwell_time: 342949
None
Length of DataFrame: 2081199
Remove threshold is 150
Remove threshold 150 is done!
number of 

# 3. extract single learner 

In [29]:
# bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment12/2012-2013-data-with-predictions-4-final.csv'
# log = pd.read_csv(bath_path, encoding = "utf-8",low_memory=False)

remove_thres = [50, 100, 150, 200]
users = []
users_log = log.groupby(['user_id', 'skill_id']).size().reset_index(name='count')

for thres in remove_thres:
    users.append(users_log[users_log['count']>thres])

In [31]:
users[0]

,user_id,skill_id,count
237,78401,27.0,53
269,78401,310.0,68
270,78401,311.0,88
277,78401,338.0,73
346,78409,338.0,51
...,...,...,...
371026,224210,311.0,67
372377,224558,34.0,53
372602,224622,34.0,97
373789,225212,34.0,59


In [53]:
######################################## single_user_single_exercise
interested_col = [
    'user_id', 'problem_id', 'skill_id', 'start_time', 'end_time', 'correct', 
    'skill', 'problem_type', 'answer_text', 
]
useful_log = log[interested_col]
remove_thres = [50, 100, 150, 200]

users_logs_threshold = []
for i in range(len(remove_thres)):
    thres = remove_thres[i]
    users_with_threshold = users[i]
    
    single_skill = {col: [] for col in interested_col}

    sk_dfs = []
    print('With threshold {} there are {} logs'.format(thres, len(users_with_threshold)))
    for j in tqdm(range(len(users_with_threshold))):
        id = users_with_threshold.user_id.iloc[j]
        ex = users_with_threshold.skill_id.iloc[j]

        single_user_exercise = useful_log.loc[(useful_log.user_id == id) & (useful_log.skill_id == ex)]
        single_user_exercise = single_user_exercise.sort_values('start_time')[:thres]

        sk_dfs.append(single_user_exercise)
    users_logs_threshold.append(sk_dfs)
    
# single_user_single_exercise = pd.DataFrame.from_dict(single_user_single_exercise)
# single_user_single_exercise.to_csv('single_user_single_exercise_junyi_200.csv')

With threshold 50 there are 4178 logs


100%|██████████| 4178/4178 [00:39<00:00, 106.04it/s]


With threshold 100 there are 664 logs


100%|██████████| 664/664 [00:06<00:00, 108.07it/s]


With threshold 150 there are 162 logs


100%|██████████| 162/162 [00:01<00:00, 108.12it/s]


With threshold 200 there are 60 logs


100%|██████████| 60/60 [00:00<00:00, 108.11it/s]


In [87]:
for i in range(len(users_logs_threshold)):
    test = users_logs_threshold[i]
    test = pd.concat(test, axis=0)
    thres = remove_thres[i]

    test.columns = ['user_id', 'problem_id', 'skill_id', 'timestamp', 'dwell_time',
           'correct', 'skill_text', 'problem_text', 'answer_text']
    new_id = [[i] * thres for i in range(len(users_logs_threshold[i]))]
    new_user_id = [item for sublist in new_id for item in sublist]
    test['user_id'] = new_user_id

    test['skill_id'] = pd.Categorical(test['skill_id'], categories=test['skill_id'].unique()).codes
    test['problem_id'] = pd.Categorical(test['problem_id'], categories=test['problem_id'].unique()).codes

    # Convert timestamp columns to a single timestamp value
    time_format = "%Y-%m-%d %H:%M:%S"
    start_times = pd.to_datetime(test.timestamp, format=time_format).apply(lambda x: x.timestamp())
    end_times = pd.to_datetime(test.dwell_time, format=time_format).apply(lambda x: x.timestamp())
    test['timestamp'] = (start_times + end_times) / 2
    test['dwell_time'] = end_times - start_times

    # Re-index columns
    test = test.astype({'timestamp': np.float64, 'dwell_time': np.float64, 'correct': np.float64})
    test = test.astype({'problem_id': np.int64, 'user_id': np.int64, 'skill_id': np.int64})

    # Save
    bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment12/single_skill/'
    test.to_csv(bath_path+'interactions_{}.csv'.format(thres), sep='\t', index=False)

In [ ]:
####################################

In [5]:
inter = pd.read_csv('/mnt/qb/work/mlcolab/hzhou52/kt/assistment12/interactions_50.csv',sep='\t')

In [6]:
inter

,user_id,problem_id,skill_id,timestamp,dwell_time,correct
0,0,0,0,1.346450e+09,120.0,0.0
1,1,1,1,1.346450e+09,17.0,1.0
2,1,2,1,1.346451e+09,73.0,0.0
3,0,3,0,1.346451e+09,105.0,1.0
4,1,4,1,1.346451e+09,67.0,0.0
...,...,...,...,...,...,...
2431783,10715,7435,69,1.379175e+09,3297933.0,0.0
2431784,12328,3023,45,1.379249e+09,3672604.0,0.0
2431785,11370,17009,45,1.379265e+09,4152537.0,0.0
2431786,10742,8867,81,1.379521e+09,3450799.0,0.0


In [9]:
user_wise_dict = dict()
cnt, n_inters = 0, 0
for user, user_df in inter.groupby('user_id'):
    df = user_df[:50]  # consider the first 50 interactions
    user_wise_dict[cnt] = {
        'user_id': user,
        'skill_seq': df['skill_id'].values.tolist(), 
        'correct_seq': [round(x) for x in df['correct']],
        'time_seq': df['timestamp'].values.tolist(),
        'problem_seq': df['problem_id'].values.tolist()
    }
    cnt += 1
    n_inters += len(df)

In [11]:
user_seq_df = pd.DataFrame.from_dict(user_wise_dict, orient='index')

In [17]:
skill_seq = np.stack(list(user_seq_df['skill_seq']))[:9000]

In [18]:
skill_seq

array([[ 0,  0,  0, ..., 25, 37, 30],
       [ 1,  1,  1, ...,  7,  0,  0],
       [ 2,  2,  2, ...,  9,  9,  9],
       ...,
       [68, 68, 68, ..., 22, 22, 15],
       [68, 68, 68, ..., 22, 22, 22],
       [39, 39, 39, ..., 42, 42, 42]])

In [52]:
cross = []
for i in range(30):
    for j in range(20):
        test = np.concatenate([skill_seq[:, i:i+1], skill_seq[:, 30+j:31+j]], axis=-1)
        cross.append(test)

In [53]:
len(cross)

600

In [57]:
cross = np.stack(cross, 0).reshape(-1, 2)

In [59]:
unique_cross = np.unique(cross, axis=0)

In [60]:
unique_cross.shape

(15389, 2)

In [61]:
263*263

69169

In [64]:
cross = []
for i in range(49):
    for j in range(i+1, 50):
        test = np.concatenate([skill_seq[:, i:i+1], skill_seq[:, j:j+1]], axis=-1)
        cross.append(test)

In [65]:
len(cross)

1225

In [66]:
cross = np.stack(cross, 0).reshape(-1, 2)

In [67]:
unique_cross = np.unique(cross, axis=0)

In [68]:
unique_cross.shape

(18108, 2)